# Yatai 

Yatai est un outil qui permet de déployer et opérer des services de Machine Learning sur Kubernetes. 
Il est composé de quatre composants:
- Yatai UI: Une interface utilisateur qui permet de gérer et créer les déploiements ainsi que les modèles.
- Yatai image builder: elle crée des images docker pour les APIs
- Yatai deployment: opérateur qui contrôle les déploiements
- bentoml: un package python qui offre une CLI et une librairie

In [ ]:
!pip install bentoml

On commence par lancer yatai via sspcloud, créer un token et se connecter avec la commande suivante:

In [ ]:
!bentoml login --endpoint <yatai_url> --api-token <token>

## 1- Entrainement du modèle

Dans ce tutoriel, on va reprendre l'exemple de détecteur de pizza. Un modèle doit être enregisté avec BentoML pour qu'on puisse le déployer avec yatai. Dans ce cas, on a un modèle Pytorch que l'on enregistre sous le bon format en utilisant la fonction (voir `train.py`):
````python
saved_model = bentoml.pytorch.save_model(
    args.model_name,
    trained_model,
    signatures=signatures,
    metadata=metadata,
    external_modules=[models],
)
````
- metadata : c'est un dictionnaire qui contient les métadonnées. Dans cet exemple, on enregistre la précision du modèle.
- signatures: `signatures = {"predict": {"batchable": True}}`: les signatures représentent des méthodes implémentées par le modèle et que l'on souhaite utiliser dans l'API. Le paramètre `batchable` indique que la fonction marche sur un lot de données (batch).

In [ ]:
!pip install -r requirements.txt
!python train.py

On peut lister les modèles enregistrés en utilisant l'interface en ligne de commande:

In [ ]:
!bentoml models list

Yatai offre un dépot centralisé des modèles. Il est configuré pour qu'il enregistre les modèles sur votre bucket S3. Pour exporter et importer des modèles sur yatai, on utilise les commandes: `bentoml models push/pull <model>`.

In [ ]:
!bentoml models push <model_tag>

**`model_name` est affiché avec la commande `bentoml models list`.**

## 2- Création du service

BentoML offre un framework pour créer des API afin de déployer des modèles. 
Le bloc de base dans bentoml est appelé **service**. Un service bentoml est composé des runners et des APIs. 

- Runner: Il s'agit de la méthode d'exécuter l'inférence du modèle. Dans `service.py`, on crée un runner pour le modèle avec: `model_runner = bentoml.pytorch.get("pytorch").to_runner()`
- APIs: les APIs définissent comment exposer le service. Un service peut avoir un plusieurs APIs. Une API est définie par l'entrée, la sortie et la fonction à exécuter. En décorant une fonction avec `@svc.api`, nous déclarons que la fonction doit être invoquée lorsque cette API est appelée. La fonction API est un endroit idéal pour définir votre logique de service, telle que la récupération de fonctionnalités, le pré et le post-traitement et les inférences de modèle via Runners. En appliquant le service, l'API est transformée en un endpoint HTTP. Dans cet exemple, la fonction `predict_image` sera exposé via le chemin `/predict_image`. On peut aussi spécifier le chemin de la fonction dans le décorateur: `@svc.api(route='/predict')`.

In [ ]:
!bentoml serve service:svc

In [ ]:
!curl -H "Content-Type: multipart/form-data" -F'fileobj=@samples/1.jpg;type=image/jpg' localhost:3000/predict_image

## 3- Construction de Bento

Bento est une archive de fichiers avec tout le code source, les modèles, les fichiers de données et les configurations de dépendance nécessaires pour exécuter un bentoml.Service défini par l'utilisateur, emballé dans un format standardisé.

Alors que bentoml.Service normalise la définition de l'API d'inférence, y compris la logique de service, l'initialisation des runners et l'entrée de l'API, les types de sortie, Bento standardise la manière de reproduire l'environnement requis pour exécuter un bentoml.Service en production.

Un Bento peut être créé avec la commande CLI `bentoml build` et un fichier de construction `bentofile.yaml`.

Le fichier de construction définit les dépendances et le code source nécessaires.

In [ ]:
!bentoml build .

In [ ]:
!bentoml list

On peut exporter le Bento créé à Yatai:

In [ ]:
!bentoml push <bento_name>

## 4- Déploiement

Une fois le Bento est exporté vers Yatai, il est simple de le déployer via l'interface. Dans la rubrique **deployments** de Yatai, on crée un nouveau et on le configure.
Ensuite, Yatai Image Builder lance automatiquement un pod pour construire et publier une image Docker qui correspond au Bento, en utilisant `kaniko`. (cette étape pourrait prendre quelques minutes)

Enfin, l'image construite est importée et lancée dans un autre 


In [ ]:
!kubectl get service

In [ ]:
!curl -H "Content-Type: multipart/form-data" -F'fileobj=@samples/1.jpg;type=image/jpg' <service_name>:3000/predict_image